In [14]:
import pandas as pd
import numpy as np

import os
import io
import base64
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from fastai.vision.all import *

In [15]:
# from scclib.util.image import correct_sized_images

In [16]:
data_path = Path('Z:\cloud\data\hpa-single-cell-image-classification')
path = Path('../data')

In [17]:
df = pd.read_csv(data_path/'train.csv')

In [18]:
df.head()

,ID,Label
0,5c27f04c-bb99-11e8-b2b9-ac1f6b6435d0,8|5|0
1,5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0,14|0
2,60b57878-bb99-11e8-b2b9-ac1f6b6435d0,6|1
3,5c1a898e-bb99-11e8-b2b9-ac1f6b6435d0,16|10
4,5b931256-bb99-11e8-b2b9-ac1f6b6435d0,14|0


In [19]:
labels = [str(i) for i in range(19)]
for x in labels:
    df[x] = df['Label'].apply(lambda r: int(x in r.split('|')))

In [20]:
dfs_0 = df[df['Label'] == '0'].sample(n=300, random_state=42).reset_index(drop=True)
dfs_1 = df[df['1'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_1u = df[df['Label'] == '1'].sample(n=221, random_state=42).reset_index(drop=True)
dfs_2 = df[df['Label'] == '2'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_3 = df[df['Label'] == '3'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_4 = df[df['Label'] == '4'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_5 = df[df['Label'] == '5'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_6 = df[df['6'] == 1].sample(n=500, random_state=42).reset_index(drop=True)
dfs_7 = df[df['Label'] == '7'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_8 = df[df['Label'] == '8'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_9 = df[df['9'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_9u = df[df['Label'] == '9'].sample(n=200, random_state=42).reset_index(drop=True)
dfs_10 = df[df['10'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_10u = df[df['Label'] == '10'].sample(n=200, random_state=42).reset_index(drop=True)
dfs_11 = df[df['11'] == 1].reset_index(drop=True)
dfs_12 = df[df['Label'] == '12'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_13 = df[df['Label'] == '13'].sample(n=400, random_state=42).reset_index(drop=True)
dfs_14 = df[df['Label'] == '14'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_15 = df[df['15'] == 1].reset_index(drop=True)
dfs_16 = df[df['Label'] == '16'].sample(n=350, random_state=42).reset_index(drop=True)
dfs_17 = df[df['17'] == 1].sample(n=500, random_state=42).reset_index(drop=True)
dfs_18 = df[df['18'] == 1].reset_index(drop=True)
dfs_ = [dfs_0, dfs_1, dfs_1u, dfs_2, dfs_3, dfs_4, dfs_5, dfs_6, dfs_7, dfs_8, dfs_9, dfs_9u, dfs_10, dfs_10u,
        dfs_11, dfs_12, dfs_13, dfs_14, dfs_15, dfs_16, dfs_17, dfs_18]

In [21]:
dfs = pd.concat(dfs_, ignore_index=True)
dfs.drop_duplicates(inplace=True, ignore_index=True)

In [22]:
def read_img(image_id: str, color: str, image_size: int):
    filename=f'{data_path}/train/{image_id}_{color}.png'
    img = Image.open(filename)
    # img = img.convert('RGB')
    size = (image_size, image_size)
    img = img.resize(size)
    img = allPil2tensor2(img)
    if img.max() > 255:
        img = torch.uint8(img/255)
    return img

def allPil2tensor2(image: Image.Image)->TensorImage:
    arr = np.asarray(image)
    if arr.ndim==2 : arr = np.expand_dims(arr,2)

    # Transpose width, height to height,width
    arr = np.transpose(arr, (1,0,2))

    # Move channels to the first position
    arr = np.transpose(arr, (2, 1, 0))

    return torch.from_numpy(arr)

In [23]:
def save_samples(image_size: int):
    all_cells = []
    # with zipfile.ZipFile(path/'cells.zip', 'w') as img_out:
    num_files = len(dfs)
    for idx in tqdm(range(num_files)):
        image_id = dfs.iloc[idx].ID
        labels = dfs.iloc[idx].Label
        red = read_img(image_id, "red", image_size)
        green = read_img(image_id, "green", image_size)
        blue = read_img(image_id, "blue", image_size)

        stacked_image = np.stack([blue, green, red], axis=1)
        stacked_image = stacked_image[0,:,:,:]

        stacked_image = np.transpose(stacked_image, (1,2,0))
        fname = f'{image_id}.jpeg'

        # im = cv2.imencode('.jpg', stacked_image)
        im = Image.fromarray(stacked_image, "RGB")

        in_mem_file = io.BytesIO()
        im.save(path/f'train/{fname}', format="JPEG")
        # in_mem_file.seek(0)
        # img_bytes = in_mem_file.read()
        #
        # base64_encoded_bytes = base64.b64encode(img_bytes)
        # im = base64_encoded_bytes.decode('ascii')
        # print(im)
        # buf = io.BytesIO(base64.b64decode(im))
        # img = Image.open(buf)
        # img.show()
        #
        # return

        # img_out.writestr(fname, im)

        all_cells.append({
            'image_id': image_id,
            'image_labels': labels
        })

    cell_df = pd.DataFrame(all_cells)
    cell_df.to_csv(path/'cell_df.csv', index=False)
    print(cell_df.head())


In [24]:
save_samples(460)

 12%|█▏        | 958/7960 [03:59<29:11,  4.00it/s]


KeyboardInterrupt: 